# Installation

There are some libraries that we need and with which the code was tested

* **taxii2-client**: To talk to the taxii server
* **stix2**: To handle the stix2 encapsulated data
* **networkx**: To create and work with graphs
* **bokeh**: To visualize graphs

In [1]:
%pip install taxii2-client==2.3.0
%pip install stix2==3.0.1
%pip install networkx==2.6.3
%pip install bokeh==2.3.3
%pip install pandas==1.3.5

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 4.2 MB/s eta 0:00:00a 0:00:01m
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.8 MB/s eta 0:00:00
Using cached idna-3.6-py3-none-any.whl (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 2.3 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 371.4 kB/s eta 0:00:00a 0:00:01
   ━

# Setup

We lower the debugging level to only show critical messages.
We use the following URL to work with:
```https://cti-taxii.mitre.org/taxii/```

In [1]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)
server = Server("https://cti-taxii.mitre.org/taxii/")

# Retrieve all the data

Let's get all the available collections and select the collection that we want to work with

In [2]:
api_root = server.api_roots[0]
for idx, collection in enumerate(api_root.collections):
    print(f"[{idx}] {collection.title} -> {collection.description}")

[0] Enterprise ATT&CK -> This data collection holds STIX objects from Enterprise ATT&CK
[1] PRE-ATT&CK -> This data collection holds STIX objects from PRE-ATT&CK
[2] Mobile ATT&CK -> This data collection holds STIX objects from Mobile ATT&CK
[3] ICS ATT&CK -> This data collection holds STIX objects from ICS ATT&CK


Make a conscious decision on the collection we want to work with

In [3]:
MITRE_COLLECTION_ID = api_root.collections[3].id

For the selected collection, actually get the te collection

In [4]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
MITRE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MITRE_COLLECTION_ID + "/")
MC_DATA = TAXIICollectionSource(MITRE_COLLECTION)

Also get the mitigation (course of action), detections and technique data. We retreive it as a mapping of unique IDs and human readable names

In [5]:
MC_MITIGATIONS = MC_DATA.query(Filter("type", "=", "course-of-action"))
mc_mitigation_mapping = {}
for x in MC_MITIGATIONS:
  mc_mitigation_mapping[x['id']] = x['name']

In [6]:
MC_TECHNIQUES = MC_DATA.query(Filter("type", "=", "attack-pattern"))
mc_technique_mapping = {}
for x in MC_TECHNIQUES:
  mc_technique_mapping[x['id']] = x['name']

In [7]:
MC_DETECTIONS = MC_DATA.query(Filter("type", "=", "x-mitre-data-component"))
mc_detection_mapping = {}
for x in MC_DETECTIONS:
  mc_detection_mapping[x['id']] = x['name']

Let's now get the relationship between course of actions (coa), detection sources and techniques. We also put the data into a panda dataframe and just for fun we ad the human readable names as well.

In [8]:
import pandas as pd

MC_RELATIONS = MC_DATA.query(Filter("type","=","relationship"))

mc_structured_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx,x in enumerate(MC_RELATIONS):
  if 'relationship_type' in x:
    if x['relationship_type'] == 'mitigates':
      mc_structured_data['Source'].append(x['source_ref'])
      mc_structured_data['Destination'].append(x['target_ref'])
      mc_structured_data['source_human'].append(mc_mitigation_mapping[x['source_ref']])
      mc_structured_data['target_human'].append(mc_technique_mapping[x['target_ref']])
    elif x['relationship_type'] == 'detects':
      mc_structured_data['Source'].append(x['source_ref'])
      mc_structured_data['Destination'].append(x['target_ref'])
      mc_structured_data['source_human'].append(mc_detection_mapping[x['source_ref']])
      mc_structured_data['target_human'].append(mc_technique_mapping[x['target_ref']])


df = pd.DataFrame(data=mc_structured_data)
df.head()

,Source,Destination,source_human,target_human
0,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--fc5fda7e-6b2c-4457-b036-759896...,Human User Authentication,Modify Program
1,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--09a61657-46e1-439e-b3ed-3e4556...,Human User Authentication,Modify Controller Tasking
2,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--097924ce-a9a9-4039-8591-e0deed...,Human User Authentication,Modify Parameter
3,course-of-action--e0d38502-decb-481d-ad8b-b8f0...,attack-pattern--fc5fda7e-6b2c-4457-b036-759896...,Authorization Enforcement,Modify Program
4,course-of-action--e0d38502-decb-481d-ad8b-b8f0...,attack-pattern--09a61657-46e1-439e-b3ed-3e4556...,Authorization Enforcement,Modify Controller Tasking


# Visualizing all the data

A lot of thanks to this information resource ```https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html``` excellent explanation and walkthrough on visualizing graph data with bokeh.

In [10]:
# https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html
from bokeh.io import output_notebook, show, save
# required to display visuals inline
output_notebook()

Loading BokehJS ...

In [11]:
from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx


DG = nx.from_pandas_edgelist(df, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}
for node in DG.nodes:
  if node.startswith('attack-pattern'):
    node_types[node] = 0
  elif node.startswith('course-of-action'):
    node_types[node] = 1
  else:
    node_types[node] = 2

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

# Visualize single techniques



In [12]:
from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx

# display all rows (be careful)
pd.set_option('display.max_rows', None)
# avoid line breaks
pd.set_option('display.expand_frame_repr', False)

# Techniques of pipedream/incontroller
#df_filtered = df[df['target_human'].isin(['Remote Services','Command-Line Interface','Execution through API','Scripting','User Execution',
#                                          'System Firmware','Valid Accounts','Exploitation for Privilege Escalation','Rootkit','Network Sniffing',
#                                          'Remote System Discovery','Remote System Information Discovery','Default Credentials','Lateral Tool Transfer',
#                                          'Program Download','Remote Services','Valid Accounts','Detect Operating Mode','Point & Tag Identification',
#                                          'Program Upload'])]

df_filtered = df[df['target_human'].isin(['Default Credentials','Valid Accounts'])]
print(df_filtered)
DG = nx.from_pandas_edgelist(df_filtered, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}
for node in DG.nodes:
  if node.startswith('attack-pattern'):
    node_types[node] = 0
  elif node.startswith('course-of-action'):
    node_types[node] = 1
  else:
    node_types[node] = 2

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

                                                Source                                        Destination                    source_human         target_human
37   x-mitre-data-component--39b9db72-8b48-4595-a18...  attack-pattern--cd2c76a4-5e23-4ca5-9c40-d5e060...          Logon Session Metadata       Valid Accounts
91   x-mitre-data-component--9ce98c86-8d30-4043-ba5...  attack-pattern--cd2c76a4-5e23-4ca5-9c40-d5e060...          Logon Session Creation       Valid Accounts
95   x-mitre-data-component--a953ca55-921a-44f7-9b8...  attack-pattern--cd2c76a4-5e23-4ca5-9c40-d5e060...     User Account Authentication       Valid Accounts
202  x-mitre-data-component--3772e279-27d6-477a-9fe...  attack-pattern--8bb4538f-f16f-49f0-a431-70b544...         Network Traffic Content  Default Credentials
219  x-mitre-data-component--9ce98c86-8d30-4043-ba5...  attack-pattern--8bb4538f-f16f-49f0-a431-70b544...          Logon Session Creation  Default Credentials
278  course-of-action--3992ce42-43e9-4bea-b8db

## ICS visualization testing

After this block we are experimenting with some matching of ICS attacks to their corresponding techniques and mitigations

In [16]:
import pandas as pd
from itertools import chain

def get_software(src):
    return list(chain.from_iterable(
        src.query(f) for f in [
            Filter("type", "=", "tool"),
            Filter("type", "=", "malware")
        ]
    ))

MC_SOFTWARE = get_software(MC_DATA)
MC_RELATIONS = MC_DATA.query(Filter("type","=","relationship"))
mc_structured_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx, x in enumerate(MC_RELATIONS):
    if 'relationship_type' in x:
        if x['relationship_type'] == 'detects':
            mc_structured_data['Source'].append(x['source_ref'])
            mc_structured_data['Destination'].append(x['target_ref'])
            mc_structured_data['source_human'].append(mc_detection_mapping[x['source_ref']])
            mc_structured_data['target_human'].append(mc_technique_mapping[x['target_ref']])
# print(MC_RELATIONS)
# print(MC_SOFTWARE)

df = pd.DataFrame(data=mc_structured_data)
df.tail()

KeyError: 'source'

# References

* https://melaniewalsh.github.io/Intro-Cultural-Analytics/welcome.html
* Attack CTI
	* https://attackcti.com/intro.html
	* https://github.com/OTRF/ATTACK-Python-Client/blob/f96f9d6094afc657682ccbc1988bae5db39552e9/attackcti/attack_api.py
* MITRE Stix / Taxii
	* https://github.com/mitre/cti/blob/master/USAGE.md
	* https://github.com/mitre-attack/attack-stix-data
	* https://github.com/oasis-open/cti-python-stix2